### Sample Insurance Company Fund mapping

#### Data type: Time series

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from sklearn.linear_model import HuberRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import csv
import os
from pyfinance.ols import PandasRollingOLS
import statsmodels.api as sm

#### Data pre processing

In [ ]:
#get a list of funds to run regression
fund_SECID = list(set(funds['SECID'].tolist()))

#get tradeable index data such as S&P returns, etc
indices_data = pd.read_excel(os.path.join(os.path.dirname(__file__), "index_data.xlsx"),engine='openpyxl')

#get mutual fund returns
#regression assumes underlying data is stationary, so we need to convert price levels to returns for modeling
NAV = pd.read_excel(os.path.join(os.path.dirname(__file__), "mutual_fund_prices.xlsx"),engine='openpyxl')

#compile a large data set with both tradeable index data and mutual fund data
#dates will also be aligned
data=pd.merge(NAV, indices_data, on=None, left_index=True, right_index=True, how='left')

#### Modeling (simplified)

In [ ]:
for ind in fund_SECID: #loop through all underlying mutual funds one at a time
        indices_fund = ['SPX_TOT_RETURN'] 
        indices_fund.append(ind) #append target variable (mutual fund return)
        baseline = data[indices_fund]
        baseline_return = pd.DataFrame(baseline.resample("W-MON").ffill().pct_change().dropna()) #calculate weekly returns
            
        x = baseline_return[indices]
        y = baseline_return[ind]

        betas = list()

        xtrain = pd.DataFrame(x[:-12])
        xtest = pd.DataFrame(x[-12:]) #last quarter of return is the holdout set
        ytrain = pd.DataFrame(y[:-12])
        ytest = pd.DataFrame(y[-12:]) #last quarter of return is the holdout set
        
        #Train model
        elasticCV = ElasticNetCV(cv=5) # ElasticNet regularized regression with cross validation 
        enet_model = elasticCV.fit(xtrain, ytrain) 

        #Test model
        y_pred = enet_model.predict(xtest.values)
        y_fitted = enet_model.predict(xtrain.values)
     
        betas.append(output)

        results=results.append(betas)

results.to_csv('results.csv')